# Parsing [avto.ru](https://auto.ru/)

In [2]:
from bs4 import BeautifulSoup
import pandas as pd    
import requests 
from pprint import pprint  

import datetime, time

# from fake_useragent import UserAgent
# headers={'User-Agent': UserAgent().chrome}

pd.set_option('display.max_columns', None)

from time import sleep
from tqdm.notebook import tqdm, tqdm_notebook

import json

## Получаем ссылки на объявления по брендам

In [9]:
response = requests.get('https://auto.ru/', headers={'User-Agent': 'Mozilla/5.0'})
# Убедимся, что мы успешно получили ответ     
print('Статуст запроса:',response.status_code)    
  
page = BeautifulSoup(response.content, 'html.parser')  

# Получаем список всех марок для которых продаются авто
avto_brand = page.find_all('div', attrs = {'class':'IndexMarks__col'})
# print(avto_brand_list)
# print(len(avto_brand))

avto_brand_list = []
for brand in avto_brand:
    for link in brand.find_all('a'):
        avto_brand_list.append([
            link.find('div', attrs = {'class':'IndexMarks__item-name'}).text.strip(), 
            int(link.find('div', attrs = {'class':'IndexMarks__item-count'}).text), 
            link.attrs['href'].strip()
            ])
        # print( link.attrs['href'])

avto_brand_list

# Созданим датасет ссылок для краулинга
df_avto_brand = pd.DataFrame(avto_brand_list,
                    columns=['brand_name', 'brand_amount', 'brand_url'])

display(df_avto_brand)
display(df_avto_brand.brand_amount.sum())

Статуст запроса: 200


,brand_name,brand_amount,brand_url
0,LADA (ВАЗ),64795,https://auto.ru/cars/vaz/all/
1,Audi,11204,https://auto.ru/cars/audi/all/
2,BMW,16540,https://auto.ru/cars/bmw/all/
3,Chery,3595,https://auto.ru/cars/chery/all/
4,Chevrolet,11679,https://auto.ru/cars/chevrolet/all/
5,Citroen,2232,https://auto.ru/cars/citroen/all/
6,Daewoo,3510,https://auto.ru/cars/daewoo/all/
7,Datsun,1229,https://auto.ru/cars/datsun/all/
8,Ford,11294,https://auto.ru/cars/ford/all/
9,Geely,2219,https://auto.ru/cars/geely/all/


346627

## ## Получаем ссылки на объявления конкретной марки

In [10]:
df_avto_brand = df_avto_brand[df_avto_brand.index == 31] # Для теста берем 31 бренд Volvo - 4072 объявлений
brand_model_list = []

with tqdm(total=df_avto_brand.shape[0]) as pbar:
    print()
    for index, brend_row in df_avto_brand.iterrows():
        
        response = requests.get(brend_row.brand_url, headers={'User-Agent': 'Mozilla/5.0'})
        # print('Статуст запроса:',response.status_code)    

        # announcement_amount = int(''.join(
        #     filter(str.isdigit, page.find('span', attrs = {'class':'ButtonWithLoader__content'}).text)))

        # print(announcement_amount)

        page = BeautifulSoup(response.content, 'html.parser')  

        # brand_model = page.find_all('div', attrs = {'class':['ListingPopularMMM-module__item']})
        brand_model = page.findAll(lambda tag: tag.name == 'div' and 
        tag.get('class') == ['ListingPopularMMM-module__item'])

        if brand_model:
            for model in brand_model:
                # print("aaa", model)
                link = (model.a)
                # print("Qqq", link)
                amount = int(model.div.text) if model.div else None,
                
                # pprint(link[0].text)
                brand_model_list.append([brend_row.brand_name, link.text.strip(), amount[0], link.attrs['href'].strip()])
        else:
            print("brand_model - пуст")

        # pbar.write(f" {brend_row.brand_name}: {index+1}/{df_avto_brand.shape[0]}")
        pbar.write(f" {brend_row.brand_name}")
        # pbar.set_description(f" {brend_row.brand_name}")
        
        pbar.update(1)
        # sleep(1)

# brand_model_list

# Созданим датасет ссылок для краулинга
df_brand_model = pd.DataFrame(brand_model_list,
                columns=['brand_name', 'model_name', 'model_amount', 'model_url'])

display(df_brand_model)
display(df_brand_model.info())

  0%|          | 0/1 [00:00<?, ?it/s]


 Volvo


,brand_name,model_name,model_amount,model_url
0,Volvo,S40,4,https://auto.ru/cars/volvo/s40/all/
1,Volvo,S60,12,https://auto.ru/cars/volvo/s60/all/
2,Volvo,S80,10,https://auto.ru/cars/volvo/s80/all/
3,Volvo,S90,75,https://auto.ru/cars/volvo/s90/all/
4,Volvo,V40 Cross Country,14,https://auto.ru/cars/volvo/v40_cc/all/
5,Volvo,V60 Cross Country,4,https://auto.ru/cars/volvo/v60_cross_country/all/
6,Volvo,V90 Cross Country,17,https://auto.ru/cars/volvo/v90_cross_country/all/
7,Volvo,XC40,34,https://auto.ru/cars/volvo/xc40/all/
8,Volvo,XC60,230,https://auto.ru/cars/volvo/xc60/all/
9,Volvo,XC70,12,https://auto.ru/cars/volvo/xc70/all/


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand_name    11 non-null     object
 1   model_name    11 non-null     object
 2   model_amount  11 non-null     int64 
 3   model_url     11 non-null     object
dtypes: int64(1), object(3)
memory usage: 480.0+ bytes


None

In [11]:
# Объединяем бренды с марками в общий датасет и если их много то сохраняем в промежуточный файл
df_avto_announcement_link = df_avto_brand.merge(df_brand_model, on='brand_name', how='left')
df_avto_announcement_link.head(5)
# df_avto_announcement_link.to_csv('avto_announcement_link.csv', index=False)

,brand_name,brand_amount,brand_url,model_name,model_amount,model_url
0,Volvo,4074,https://auto.ru/cars/volvo/all/,S40,4,https://auto.ru/cars/volvo/s40/all/
1,Volvo,4074,https://auto.ru/cars/volvo/all/,S60,12,https://auto.ru/cars/volvo/s60/all/
2,Volvo,4074,https://auto.ru/cars/volvo/all/,S80,10,https://auto.ru/cars/volvo/s80/all/
3,Volvo,4074,https://auto.ru/cars/volvo/all/,S90,75,https://auto.ru/cars/volvo/s90/all/
4,Volvo,4074,https://auto.ru/cars/volvo/all/,V40 Cross Country,14,https://auto.ru/cars/volvo/v40_cc/all/


In [12]:
# df_avto_announcement_link = pd.read_csv('avto_announcement_link.csv')
# df_avto_announcement_link.head(5)

## Получаем ссылки на конкретное объявление

In [13]:
def get_brand_model_announcement(model_url, message):
    '''
    В страничном режиме сайт показывает не болле 99 страниц по 37 объявлении, поэтому лучше парсить 
    модель каждой марки отдельно чтобы получить в лучшем случае для одной модели 99*37=3 663 объявлений
    '''
    announcement_list = [] # список ссылок на объявления


    for page in  tqdm_notebook(range(1, 100), desc=message):
        url = f"{model_url}?output_type=table&page={page}"

        # Произведем замену в url пути all на used - это даст возможность смотреть только объявления подержаных машин
        url = url.replace('/all/', '/used/') 
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

        page = BeautifulSoup(response.content, 'html.parser')  
        # pprint(page)
        # page_announcement = page.findAll(lambda tag: tag.name == 'div' and 
        # tag.get('class') == ['ListingCars-module__listingItem'])
        # page_announcement = page.find_all('div', attrs = {'class':['ListingCars-module__listingItem']})
        page_announcement = page.find_all('div', attrs = {'class':['ListingItemSequential-module__container']})
       
        if page_announcement: # если объявление не закончились
            for announcement in page_announcement:
                link = announcement.find('a', attrs = {'class':['Link', 'ListingItemTitle__link']})
                announcement_list.append([model_url, link.attrs['href'].strip()])
        else:
            break

            # pprint(link.attrs['href'].strip())
        # pprint(page_announcement)
        # print(len(page_announcement))
        # print('Статуст запроса:',response.status_code)
    
    return announcement_list
  

In [15]:
df_copy = df_avto_announcement_link.copy()[:2]  # Для теста берем перых бренд две модели

pbar = tqdm_notebook(total=df_copy.shape[0])

all_brend_announcement_list = []
for index, announcement_row in  df_copy.iterrows():
    message = f"{index+1} {announcement_row.brand_name} - {announcement_row.model_name}"
    pbar.set_description(message)
    pbar.update()
    
    announcement_list = get_brand_model_announcement(announcement_row.model_url, message)
    # pbar.write(f" {announcement_row.brand_name} - {announcement_row.model_name}")
    all_brend_announcement_list.extend(announcement_list)
    
    # display(announcement.head())
    # display(df_brand_model.info())
    # return df_announcement

    # print(f" {announcement_row.brand_name} - {announcement_row.model_name}")
    # sleep(0.1)

    # Созданим датасет ссылок для краулинга и каждый раз будем его перезаписывать
    df_announcement = pd.DataFrame(all_brend_announcement_list, columns=['model_url', 'announcement_url'])
    # display(df_announcement.head())
    # display(df_announcement.info())

    # Объединим и сохраним результат
    df_all_brend_announcement_link = df_avto_announcement_link.merge(df_announcement, on='model_url', how='left')
    # display(df_all_brend_announcement_link.head(5))

  0%|          | 0/2 [00:00<?, ?it/s]

1 Volvo - S40:   0%|          | 0/99 [00:00<?, ?it/s]

2 Volvo - S60:   0%|          | 0/99 [00:00<?, ?it/s]

In [17]:
# При необходимости сохраняем в файл
# df_all_brend_announcement_link.to_csv('all_brend_announcement_link.csv', index=False)
# df_avto_announcement_link_completed = pd.read_csv('all_brend_announcement_link (completed).csv')
df_avto_announcement_link_completed = df_all_brend_announcement_link
df_avto_announcement_link_completed.head(5)

,brand_name,brand_amount,brand_url,model_name,model_amount,model_url,announcement_url
0,Volvo,4074,https://auto.ru/cars/volvo/all/,S40,4,https://auto.ru/cars/volvo/s40/all/,https://auto.ru/cars/used/sale/volvo/s40/11042...
1,Volvo,4074,https://auto.ru/cars/volvo/all/,S40,4,https://auto.ru/cars/volvo/s40/all/,https://auto.ru/cars/used/sale/volvo/s40/11031...
2,Volvo,4074,https://auto.ru/cars/volvo/all/,S40,4,https://auto.ru/cars/volvo/s40/all/,https://auto.ru/cars/used/sale/volvo/s40/11031...
3,Volvo,4074,https://auto.ru/cars/volvo/all/,S40,4,https://auto.ru/cars/volvo/s40/all/,https://auto.ru/cars/used/sale/volvo/s40/11047...
4,Volvo,4074,https://auto.ru/cars/volvo/all/,S40,4,https://auto.ru/cars/volvo/s40/all/,https://auto.ru/cars/used/sale/volvo/s40/11046...


## Парсим каждое объявление в несколько потоков

In [19]:
def get_announcement_value_by_name(element, name):
    card_info_li = element.find('li', attrs = {'class':[f'CardInfoRow {name}']})
    # pprint(card_info_li)
    if card_info_li:
        span_list = card_info_li.find_all('span', attrs={'class':['CardInfoRow__cell']})
        # pprint(span_list)
        if span_list and len(span_list) > 1:
            return span_list[1].text.strip()


def announcement_info(item_dict):
    '''Получает информацию с обьявления'''
    # timeout=5
    announcement_url = item_dict['car_url']

    response = requests.get(announcement_url, headers={'User-Agent': 'Mozilla/5.0'})
    page = BeautifulSoup(response.content, 'html.parser')  

    card_info_ul = page.find('ul', attrs = {'class':['CardInfo']})

    # дернем json содержащий дополнительную информацию
    element_addition_info_json = page.find('script', attrs={'id': 'initial-state'})
    
    # чтобы не мучаться возьмем только первый нужный нам тэг 'script', хотя на странице их 3
    element_json = page.find('script', attrs={'type': 'application/ld+json'})
    if element_json and card_info_ul and element_addition_info_json:
        # pprint(element_json)
        element_json = json.loads(element_json.string)
        element_addition_info_json = json.loads(element_addition_info_json.string)

        item_dict['bodyType'] = element_json['bodyType']
        # item_dict['brand'] # записано раньше
        # item_dict['car_url'] # записано раньше  
        item_dict['color'] = element_json['color']
        
        try:
             item_dict['complectation_dict'] = element_addition_info_json['card']['vehicle_info']['complectation']
        except:
            print(f"{announcement_url} не найден: element_addition_info_json['card']['vehicle_info']['complectation']")

        item_dict['description'] = element_json['description']
        item_dict['engineDisplacement'] = element_json['vehicleEngine']['engineDisplacement']
        item_dict['enginePower'] = element_json['vehicleEngine']['enginePower']
       
        try:
             item_dict['equipment_dict'] = element_addition_info_json['card']['vehicle_info']['equipment']
        except:
            print(f"{announcement_url} не найден: element_addition_info_json['card']['vehicle_info']['equipment']")

        item_dict['fuelType'] = element_json['fuelType']

        try:
             item_dict['image'] = element_json['image'] # не нужна
        except:
            print(f"{announcement_url} не найден: item_dict['image']")
        
        item_dict['mileage'] = ''.join(filter(str.isdigit, get_announcement_value_by_name(card_info_ul, 'CardInfoRow_kmAge')))
        
        try:
            item_dict['modelDate'] = element_json['modelDate']
        except:
            print(f"{announcement_url} не найден: element_json['modelDate']")
       
       
        try:
             item_dict['model_info'] = element_addition_info_json['card']['vehicle_info']['model_info']
        except:
            print(f"{announcement_url} не найден: element_addition_info_json['card']['vehicle_info']['model_info']")

        # item_dict['model_name'] # записано раньше
        item_dict['name'] = element_json['name']
        item_dict['numberOfDoors'] = element_json['numberOfDoors']
        

        # datetime.datetime.fromtimestamp(item_dict['parsing_unixtime'])
        item_dict['parsing_unixtime'] = int(time.time()) 
        item_dict['priceCurrency'] = element_json['offers']['priceCurrency']
        item_dict['productionDate'] = element_json['productionDate']
        item_dict['sell_id'] = element_json['offers']['url'].rsplit('/', 2)[1].split('-')[0].strip()
       
        try:
             item_dict['super_gen'] = element_addition_info_json['card']['vehicle_info']['super_gen']
        except:
            print(f"{announcement_url} не найден: element_addition_info_json['card']['vehicle_info']['super_gen']")

        item_dict['vehicleConfiguration'] = element_json['vehicleConfiguration']
        item_dict['vehicleTransmission'] = element_json['vehicleTransmission']
        try:
            item_dict['vendor'] = element_addition_info_json['card']['vehicle_info']['vendor']
        except:
            print(f"{announcement_url} не найден: element_addition_info_json['card']['vehicle_info']['vendor']")
        
        item_dict['Владельцы'] = get_announcement_value_by_name(card_info_ul, 'CardInfoRow_ownersCount')
        item_dict['Владение'] = get_announcement_value_by_name(card_info_ul, 'CardInfoRow_owningTime')
        item_dict['ПТС'] = get_announcement_value_by_name(card_info_ul, 'CardInfoRow_pts')
        item_dict['Привод'] = get_announcement_value_by_name(card_info_ul, 'CardInfoRow_drive')
        item_dict['Руль'] = get_announcement_value_by_name(card_info_ul, 'CardInfoRow_wheel')
        item_dict['Состояние'] = get_announcement_value_by_name(card_info_ul, 'CardInfoRow_state')
        item_dict['Таможня'] = get_announcement_value_by_name(card_info_ul, 'CardInfoRow_customs')
        
        try:
            item_dict['price'] = element_json['offers']['price']
        except:
            print(f"{announcement_url} не найден: price")

    # pprint(item_dict)
    return [value for key, value in item_dict.items()]

In [21]:
columns = [
    'bodyType', 'brand', 'car_url', 'color', 'complectation_dict', 
    'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
    'fuelType', 'image', 'mileage', 'modelDate',
    'model_info', 'model_name', 'name', 'numberOfDoors',
    'parsing_unixtime', 'priceCurrency', 'productionDate', 'sell_id',
    'super_gen', 'vehicleConfiguration', 'vehicleTransmission', 'vendor', 
    'Владельцы', 'Владение', 'ПТС', 'Привод', 
    'Руль', 'Состояние', 'Таможня', 'price',
]
item_dict = dict((key, None) for key in columns)

df_copy = df_avto_announcement_link_completed.copy()#[83000+80000:]

In [22]:
from threading import Thread, current_thread
# from Queue import Queue
import queue
theard_count = 15

pbar = tqdm_notebook(total=df_copy.shape[0])

all_announcement_info_list = []

def run(queue, result_queue):
    '''
        queue - очередь c url
        result_queue - очередь с ответами
    '''
    # Цикл продолжается пока очередь задач не станет пустой
    while not queue.empty():
        # получаем первую задачу из очереди
        item_dict1 = queue.get_nowait()
        host = item_dict1['car_url']
        # print(f'{ host } проверка в потоке { current_thread() }')
        
        # парсим объявление
        announcement_info1 = announcement_info(item_dict1)

        # status = check_url(host)
        # сохраняем результат для дальнейшей обработки

        result_queue.put_nowait(announcement_info1)

        all_announcement_info_list.append(announcement_info1)
        # pprint(all_announcement_info_list)
        if (len(all_announcement_info_list)+1) % 1000 == 0:
        # Созданим датасет и будем сохранять его каждые 1000 итераций
            df = pd.DataFrame(all_announcement_info_list, columns=columns)
            display(df.sample(1))
            df.to_csv(f'{len(all_announcement_info_list)} 20210719_all_announcement_info_theard.csv', index=False)


        # сообщаем о выполнении полученной задачи
        queue.task_done()
        pbar.update()
        # print(f'{ host } закончено в потоке { current_thread() }\n')

    # print(f'{current_thread()} закрытие потока\n')

# MAIN

all_announcement_info_list1 = []

def main():
    start_time = time.time()

    # Для получения задач и выдачи результата используем очереди
    queue1 = queue.Queue()
    result_queue1 = queue.Queue()

    #Сначала загружаем все URL из датасет в очередь задач
    for index, announcement_row in  df_copy.iterrows():
        item_dict1 = item_dict.copy()
        item_dict1['brand'] = announcement_row.brand_name
        item_dict1['car_url'] = announcement_row.announcement_url
        item_dict1['model_name'] = announcement_row.model_name
        queue1.put(item_dict1)


    # Затем запускаем необходимое количество потоков
    for i in range(theard_count):
        thread = Thread(target=run, args=(queue1, result_queue1))
        thread.daemon = True
        thread.start()

    # И ждем, когда задачи будут выполнены    
    queue1.join()

    # После чего пишем результаты в файлы

    while not result_queue1.empty():
        announcement_info1 = result_queue1.get_nowait()
        all_announcement_info_list1.append(announcement_info1)
        df = pd.DataFrame(all_announcement_info_list1, columns=columns)
        df.to_csv('20210719_all_announcement_info_theard (complete all).csv', index=False)

    print("Завершено:", time.time() - start_time)

main()

  0%|          | 0/416 [00:00<?, ?it/s]

https://auto.ru/cars/used/sale/volvo/s60/1104751947-668259de/ не найден: element_addition_info_json['card']['vehicle_info']['complectation']
https://auto.ru/cars/used/sale/volvo/s60/1104751947-668259de/ не найден: element_addition_info_json['card']['vehicle_info']['vendor']
https://auto.ru/cars/used/sale/volvo/s60/1104624075-85cea8a6/ не найден: element_addition_info_json['card']['vehicle_info']['complectation']
https://auto.ru/cars/used/sale/volvo/s60/1104624075-85cea8a6/ не найден: element_addition_info_json['card']['vehicle_info']['vendor']
https://auto.ru/cars/used/sale/volvo/s60/1104624075-85cea8a6/ не найден: price
https://auto.ru/cars/used/sale/volvo/s60/1104703444-7d943b78/ не найден: price


Exception in thread Exception in thread Thread-9Thread-12:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
:
Exception in thread Exception in thread Thread-18Traceback (most recent call last):
Thread-17:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
        self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
self.run()
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
        self.run()self._target(*self._args, **self._kwargs)

  File "<ipython-input-22-0dac26be271e>", line 23, in run
  File "C:\ProgramData\Anaconda3\lib\t

In [ ]:
# Спарсим каждое объявление

# pbar = tqdm_notebook(total=df_copy.shape[0])

# all_announcement_info_list = []
# for index, announcement_row in  df_copy.iterrows():
    
#     # занесем уже известные значения словарь
#     item_dict['brand'] = announcement_row.brand_name
#     item_dict['car_url'] = announcement_row.announcement_url
#     item_dict['model_name'] = announcement_row.model_name

#     all_announcement_info_list.append(announcement_info(item_dict))
#     # display(all_announcement_info_list)
#     pbar.update()
#     # sleep(0.1)

#     if (index+1) % 1000 == 0:
#         # Созданим датасет и будем сохранять его каждые 1000 итераций
#         df = pd.DataFrame(all_announcement_info_list, columns=columns)
#         display(df.sample(1))
#         df.to_csv('20210719_all_announcement_info.csv', index=False)

  0%|          | 0/234086 [00:00<?, ?it/s]

KeyboardInterrupt: 